In [2]:
%pip install seafreeze
%pip install parquet
%pip install pyarrow
%pip install pandas
import numpy as np
from seafreeze import seafreeze as sf
import os as os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

  Using cached SeaFreeze-1.0.0-py3-none-any.whl (7.4 MB)
  Using cached hdf5storage-0.1.19-py2.py3-none-any.whl (53 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached parquet-1.3.1-py3-none-any.whl (24 kB)
  Using cached thriftpy2-0.5.2.tar.gz (782 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached ply-3.11-py2.py3-none-any.whl (49 kB)
  Using cached Cython-3.0.11-py2.py3-none-any.whl (1.2 MB)
  Created wheel for thriftpy2: filename=thriftpy2-0.5.2-cp39-cp39-macosx_11_0_arm64.whl size=787516 sha256=73ad8090e0771da24f14665b6568410d41d39d970e9b05633c0c3f7e066cb49f
  Stored in directory: /Users/ulajones/Library/Caches/pip/wheels/95/51/1d/d7303cb7c6b02c5793595b3138311bc81c4b5470ed7d306aa2
Successfully built thriftpy2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may nee

I initially generate the SeaFreeze model data as numpy arrays, which I convert to csv files and finally parquet tables. This seems to be the most efficient format for column-oriented data storage + retrieval. 

Everything is saved to dedicated folders and organized by thermodynamic variable. Due to the large size of the dataset (1.5 Gb), I am hosting it on Google Drive at this link: https://drive.google.com/drive/folders/1_uFESfnrcsbmZZ5INQRrxB_oK4BcsKiu?usp=drive_link 

I chose to save the data as lists of length P*T*m instead of arrays, as this allows me to keep all the data associated with each thermodynamic variable in one file. These can later be reshaped to the original PxT arrays. I filter for inf values, which we might expect around critical points, and NaN values in case I am out of range for any calculations, then deposit the data in the ai_ready folder.

The thermodynamic variables and their units are as follows: 

| Quantity  (PT and PTm)      |  Symbol in SeaFreeze  |  Unit (SI)  |
| --------------- |:---------------------:| :----------:|
| Gibbs Energy           | `G` | J/kg |
| Entropy                | `S` | J/K/kg |
| Internal Energy        | `U` | J/kg |
| Enthalpy               | `H` | J/kg |
| Helmholtz free energy  | `A` | J/kg |
| Density                |`rho`| kg/m^3 |
| Specific heat capacity at constant pressure|`Cp`| J/kg/K |
| Specific heat capacity at constant volume|`Cv`| J/kg/K |
| Isothermal bulk modulus      |`Kt`| MPa |
| Pressure derivative of the Isothermal bulk modulus|`Kp`| - |
| Isoentropic bulk modulus     |`Ks`| MPa |
| Thermal expansivity     |`alpha`| /K |
| Bulk sound speed     |`vel`| m/s |
| Solute Chemical Potential           | `mus` | J/mol |
| Solvent Chemical Potential                | `muw` | J/mol |
| Partial Molar Volume        | `Vm` | cc/mol |
| Partial Molar Heat Capacity               | `Cpm` | J/kg/K/mol |
| Apparent Heat Capacity  | `Cpa` | J/kg/K/mol |
| Apparent Volume                |`Va`| cc/mol |
| Excess Volume|`Vex`| cc/mol |
| Osmotic Coefficient|`phi`| -|
| Water Activity      |`aw`| - |
| Activity Coefficient|`gam`| - |
| Excess Gibbs Energy     |`Gex`| J/kg |

In [3]:
dir0 = '/Users/ulajones/Documents/School/ESS/ESS569F24/MLGEO2024_NaCl_EoS/data/raw'

# Define the range of pressure, temperature, and molality
P = np.arange(0.1, 1000.2, 10)
T = np.arange(240, 501, 2)
m = np.arange(0, 7.1, 0.1)

# Define the properties to compute
props = ('G', 'S', 'U', 'H', 'A', 'rho', 'Cp', 'Kt', 'Kp', 'Ks', 'alpha', 'vel', 'mus', 'muw', 'Vm', 'Cpm', 'Cpa', 'Va', 'Vex', 'phi', 'aw', 'gam', 'Gex')
data_dict = {}

for j in props: 
    # Create directories to store the data for each property
    directory_path = os.path.join(dir0, j)
    os.makedirs(directory_path, exist_ok=True)
    dir_clean = os.path.join('/Users/ulajones/Documents/School/ESS/ESS569F24/MLGEO2024_NaCl_EoS/data/clean/', j)
    os.makedirs(dir_clean, exist_ok=True)
    dir_ready = os.path.join('/Users/ulajones/Documents/School/ESS/ESS569F24/MLGEO2024_NaCl_EoS/data/ai_ready/', j)
    os.makedirs(dir_ready, exist_ok=True)
    
    combined_data = []

    # generate individual files for each molality
    for i in m:
        ptm = np.array([P, T, [i]], dtype=object)
        out = sf.getProp(ptm, 'NaClaq')
        i_str = str(i).replace('.', '_')  # Replace decimal point with underscore
        filename = f"props_{j}_{i_str}_molkg.csv"
        
        # extract data and remove unnecessary dimension
        field_data = np.squeeze(getattr(out, j), axis=2)
        
        # Save the combined data to a CSV file
        np.savetxt(os.path.join(directory_path, filename), field_data, delimiter=",", fmt='%s')
                
        # Iterate over temperature and pressure, append all data to a list
        for t_idx, temp in enumerate(T):
            for p_idx, pres in enumerate(P):
                value = field_data[p_idx, t_idx]
                combined_data.append([pres, temp, i, value])
        
    # Drop rows with NaN, inf values
    combined_data = combined_data.replace([np.inf, -np.inf], np.nan, inplace= False)
    combined_data = combined_data.dropna(inplace= False)
    
    # Save combined data to CSV files and parquet tables in the ai ready folder 
    np.savetxt(os.path.join(dir_clean, f"all_props_{j}.csv"), combined_data, delimiter=",", fmt='%s')
    header_list = ['P_Mpa', "T_K", 'm_molkg', f'{j}']
    data_dict[j] = pd.read_csv(os.path.join(dir_clean, f"all_props_{j}.csv"), names=header_list)
    pq.write_table(pa.Table.from_pandas(data_dict[j]), (os.path.join(dir_ready, f'NaCl_{j}.parquet')))

# check the files in the directory
files = os.listdir(directory_path)
print(files)

# read parquet files
for j in props: 
    dir_ready = os.path.join('/Users/ulajones/Documents/School/ESS/ESS569F24/MLGEO2024_NaCl_EoS/data/ai_ready/', j)
    pq.read_table(os.path.join(dir_ready, f'NaCl_{j}.parquet'))

/opt/anaconda3/envs/eos_env/lib/python3.9/site-packages/lbftd/statevars.py:205: RuntimeWarning: divide by zero encountered in log
  logVal = 1/R * 1/nu * (tdv.mus - gss) / gPTM[iT].astype(float) - np.log(gPTM[iM].astype(float))
/opt/anaconda3/envs/eos_env/lib/python3.9/site-packages/lbftd/statevars.py:213: RuntimeWarning: invalid value encountered in multiply
  return R*nu*gPTM[iT]*gPTM[iM]*(np.log(tdv.gam)+(1-tdv.phi))


KeyboardInterrupt: 

The model data I generated using this method are hosted here: https://drive.google.com/drive/folders/1_uFESfnrcsbmZZ5INQRrxB_oK4BcsKiu?usp=drive_link 